In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

## Fetch the vertebral dataset

In [ ]:
df = pd.read_csv('./column_3C.dat', sep = ' ')
print (np.shape(df))

In [ ]:
X = df.drop('Y', axis = 1).values
y = df.Y

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

In [ ]:
import pickle
from joblib import load, dump
# dump(clf, './vertebral_clf.joblib') 

In [ ]:
clf = load('./vertebral_clf.joblib')
#clf.score(X_test, Y_test)

In [ ]:
w1=clf.coefs_[0]
b1=clf.intercepts_[0]
w2=clf.coefs_[1]
b2=clf.intercepts_[1]

In [ ]:
wb1=[]
wb1.append(w1)
wb1.append(b1)

wb2=[]
wb2.append(w2)
wb2.append(b2)

### genetic algorithm to determine the relu size, weight size, bias size and sparsity

In [ ]:
#print(clf.score(X_test, Y_test))

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler
import hls4ml
import write_mlp_mergemult_ps as wv

In [ ]:
import blackbox as bb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import Problem
from pymoo.operators.repair.rounding import RoundingRepair
import area as ar
from importlib import reload
reload(bb)
reload(ar)

In [ ]:
import weightsharing as ws
from importlib import reload

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self,weightsbiases1,weightsbiases2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train):
        self.weightsbiases1=weightsbiases1
        self.weightsbiases2=weightsbiases2
        self.layer1=layer1
        self.layer2=layer2
        self.layer3=layer3
        self.X_test=X_test
        self.X_train=X_train
        self.Y_test=Y_test
        self.Y_train=Y_train
        #x[0]: relu_size
        #x[1]: weight size layer1
        #x[2]: bias size layer1
        #x[3]: weight size layer2
        #x[4]: bias size layer2
        #x[5]: pruning sparsity
        #x[6]: input size
        #x[7]: int relu size
        #x[8]: int w1 size
        #x[9]: int b1 size
        #x[10]: int w2 size
        #x[11]: int b2 size
        
        super().__init__(n_var=12,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([4,3,2,3,2,0,2,0,0,0,0,0]),
                         xu=np.array([8,7,4,7,4,8,4,1,2,1,2,1]),
                         vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        accuracy, weights = bb.blackbox(self.weightsbiases1,self.weightsbiases2, x[0], x[1], x[2], x[3], x[4], x[5], x[6] , x[7], x[8], x[9], x[10], x[11],  self.layer1, self.layer2, self.layer3, self.X_test, self.Y_test, self.X_train, self.Y_train)
        f1 = 1- accuracy
        f2 = ar.area(weights,x[6],x[0],x[1],x[3], x[8], x[10], self.layer1,self.layer2,self.layer3)

        out["F"] = [f1, f2]


In [ ]:
layer1=6
layer2=3
layer3=3
problem = MyProblem(wb1,wb2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train)

In [ ]:
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=60,
    sampling=IntegerRandomSampling(),
    crossover=SBX(vtype=float, repair=RoundingRepair()),
    mutation=PM( vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

In [ ]:
from pymoo.termination import get_termination

termination = get_termination("n_gen",60)

In [ ]:
%%time
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F

### plot all the solutions from the final generation

In [ ]:
from pymoo.visualization.scatter import Scatter
pop=res.pop
vals=pop.get("F")
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(vals, facecolor="none", edgecolor="red")
plot.show()

### plot the pareto solutions from the final generation

In [ ]:
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(vals, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
layer_1=6
layer_2=3
layer_3=3

In [ ]:
# import pickle
# with open("paretos_vertebral", "wb") as fp:   #Pickling
#     pickle.dump(paretos_1, fp)
# with open("costs_vertebral", "wb") as fp:   #Pickling
#     pickle.dump(costs, fp)

In [ ]:
with open("paretos_vertebral", "rb") as fp:   # Unpickling
    paretos = pickle.load(fp)
with open("costs_vertebral", "rb") as fp:   # Unpickling
    costs = pickle.load(fp)

In [ ]:
paretos

In [ ]:
import pareto_verilogs as pv
import pareto_weight_sharing_verilogs as pwsv
import pareto_clustering_verilogs as pcv
import pareto_only_quantization_verilogs as poqv
import only_pruning_verilog as opv
from importlib import reload
reload(pv)
reload(pwsv)
reload(pcv)
reload(poqv)
reload(opv)

In [ ]:
i=0
layer1=6
layer2=3
layer3=3
epochs=4
lr=0.005
opv.generate(i,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)

In [ ]:
i=0
for solution in paretos:
    print("solution is"+str(solution))
    relusize_f=solution[0]
    weight_size_f1=solution[1]
    bias_size_f1=solution[2]
    weight_size_f2=solution[3]
    bias_size_f2=solution[4]
    sparsity=solution[5]
    inputsize=solution[6]
    relusize_int=solution[7]
    w1_int=solution[8]
    b1_int=solution[9]
    w2_int=solution[10]
    b2_int=solution[11]
    model=pv.generate(i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,sparsity,inputsize,relusize_int,w1_int,b1_int,w2_int,b2_int,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)
    pcv.generate(model,i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,inputsize,relusize_int,w1_int,b1_int,w2_int,b2_int,layer1,layer2,layer3,X_train,Y_train,X_test,Y_test)
    i += 1

In [ ]:
paretos

In [ ]:
# import pickle
# with open("true_paretos_vertebral", "wb") as fp:   #Pickling
#     pickle.dump(true_paretos, fp)
# with open("true_accuracies_vertebral", "wb") as fp:   #Pickling
#     pickle.dump(accuracy_list, fp)
# with open("true_weights_vertebral", "wb") as fp:   #Pickling
#     pickle.dump(pweights_list, fp)

In [ ]:
import pickle
with open("true_paretos_vertebral", "rb") as fp:   # Unpickling
    true_paretos = pickle.load(fp)
with open("true_weights_vertebral", "rb") as fp:   # Unpickling
    true_paretos_weights = pickle.load(fp)
with open("true_accuracies_vertebral", "rb") as fp:   # Unpickling
    true_paretos_accuracies = pickle.load(fp)